In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import RobustScaler, MinMaxScaler
import gc
import os

In [3]:
train = pd.read_csv('../input/train_ffts.csv.gz')
test = pd.read_csv('../input/test_ffts.csv.gz')
submission = pd.read_csv('../input/sample_submission.csv')

In [4]:
X = train[train.columns[1:-1]].values
X_test = test[test.columns[1:]].values
Y = train.label.values

In [5]:
X.max()

79483.0

In [6]:
X.min()

-40393.74516055893

In [7]:
X_test.max()

70934.0

In [8]:
X_test.min()

-40394.43510378474

In [9]:
X /= 79483.0
X_test /= 79483.0

In [10]:
train.head()

,Unnamed: 0,column0,column1,column2,column3,column4,column5,column6,column7,column8,...,column1559,column1560,column1561,column1562,column1563,column1564,column1565,column1566,column1567,label
0,0,16649.0,-10825.730588,676.963766,2972.747410,-331.791733,-1630.186080,800.741748,80.0,145.728115,...,57.092994,164.158199,331.641365,-466.608810,-749.407401,1916.178001,-1806.720945,1555.701842,-1645.464688,1
1,1,44609.0,-19817.879430,-11899.700821,7152.010894,6192.999503,-2268.783599,-2748.228295,488.0,390.744388,...,307.033887,-118.143121,373.806105,-1765.320781,-373.047966,2168.431961,1353.943202,-2605.329830,-707.260374,0
2,2,13425.0,-13147.387496,12341.449201,-11083.963364,9490.534722,-7699.389592,5852.892616,-4080.0,2482.590550,...,-587.401256,743.532013,-779.703518,677.923513,-447.766075,125.006110,234.974359,-567.726821,812.157610,1
3,3,15025.0,-6462.372814,-6805.507220,7533.698456,946.579987,-4640.311023,1487.771136,747.0,195.898912,...,706.047167,143.044108,-964.441545,526.927652,233.579291,-416.451157,853.525244,-1700.343512,1857.337167,4
4,4,51093.0,-18393.458365,-16356.968576,3604.196760,6944.535309,678.005820,-527.521874,-492.0,-1811.020731,...,1134.303456,509.522881,-696.614075,681.470093,-974.856978,-2273.866761,2626.761876,2009.183212,-2764.059140,0


In [11]:
X

array([[ 0.20946618, -0.13620184,  0.00851709, ..., -0.02273091,
         0.01957276, -0.0207021 ],
       [ 0.56123951, -0.24933482, -0.14971379, ...,  0.01703437,
        -0.03277845, -0.00889826],
       [ 0.16890404, -0.16541131,  0.15527156, ...,  0.00295628,
        -0.00714275,  0.010218  ],
       ...,
       [ 0.39636149, -0.24301471,  0.01591474, ...,  0.09482276,
        -0.17273383,  0.05732685],
       [ 0.33190745, -0.22707595,  0.04505359, ..., -0.06421729,
         0.06926826,  0.00676997],
       [ 0.22870299, -0.17247236,  0.05447384, ...,  0.04227385,
        -0.00506188, -0.02353869]])

In [12]:
X.max()

1.0

In [13]:
X.min()

-0.5082060964050039

In [14]:
X_test.max()

0.8924424090686058

In [15]:
X_test.min()

-0.5082147767923296

In [16]:
train_oof = np.zeros((X.shape[0], 10))
test_preds = 0
train_oof.shape

(42000, 10)

In [17]:
%%time
n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(X)):
    print("Fitting fold", jj+1)
    train_features = X[train_index]
    train_target = Y[train_index]
    
    val_features = X[val_index]
    val_target = Y[val_index]
    
    model = SVC(kernel='rbf',C=10, probability=True)
    model.fit(train_features, train_target)
    val_pred = model.predict_proba(val_features)
    train_oof[val_index] = val_pred
    print("Fold accuracy:", accuracy_score(val_target, np.argmax(val_pred, axis=1)))
    test_preds += model.predict_proba(X_test)/n_splits
    del train_features, train_target, val_features, val_target
    gc.collect()

Fitting fold 1
Fold accuracy: 0.979047619047619
Fitting fold 2
Fold accuracy: 0.9821428571428571
Fitting fold 3
Fold accuracy: 0.9802380952380952
Fitting fold 4
Fold accuracy: 0.9792857142857143
Fitting fold 5
Fold accuracy: 0.9810714285714286
CPU times: user 55min 58s, sys: 535 ms, total: 55min 59s
Wall time: 55min 59s


In [18]:
print(accuracy_score(Y, np.argmax(train_oof, axis=1)))

0.9803571428571428


In [19]:
train = pd.read_csv('../input/train_hffts.csv.gz')
test = pd.read_csv('../input/test_hffts.csv.gz')


In [20]:
X = train[train.columns[1:-1]].values
X_test = test[test.columns[1:]].values
Y = train.label.values
X.max()

158966.0

In [21]:
X_test.max()

141868.0

In [22]:
X.min()

-76874.08874064806

In [23]:
X_test.min()

-77447.34682558887

In [24]:
X /= 158966.0
X_test /= 158966.0

In [25]:
%%time

train_oof = np.zeros((X.shape[0], 10))
test_preds = 0
train_oof.shape

n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(X)):
    print("Fitting fold", jj+1)
    train_features = X[train_index]
    train_target = Y[train_index]
    
    val_features = X[val_index]
    val_target = Y[val_index]
    
    model = SVC(kernel='rbf',C=10, probability=True)
    model.fit(train_features, train_target)
    val_pred = model.predict_proba(val_features)
    train_oof[val_index] = val_pred
    print("Fold accuracy:", accuracy_score(val_target, np.argmax(val_pred, axis=1)))
    test_preds += model.predict_proba(X_test)/n_splits
    del train_features, train_target, val_features, val_target
    gc.collect()
    
print(accuracy_score(Y, np.argmax(train_oof, axis=1)))

Fitting fold 1
Fold accuracy: 0.979047619047619
Fitting fold 2
Fold accuracy: 0.9817857142857143
Fitting fold 3
Fold accuracy: 0.9802380952380952
Fitting fold 4
Fold accuracy: 0.9795238095238096
Fitting fold 5
Fold accuracy: 0.9813095238095239
0.9803809523809524
CPU times: user 54min 10s, sys: 408 ms, total: 54min 10s
Wall time: 54min 10s


In [26]:
train = pd.read_csv('../input/train_rffts.csv.gz')
test = pd.read_csv('../input/test_rffts.csv.gz')

In [27]:
X = train[train.columns[1:-1]].values
X_test = test[test.columns[1:]].values
Y = train.label.values
X.max()

79483.0

In [28]:
X.min()

-40393.74516055893

In [29]:
X_test.max()

70934.0

In [30]:
X_test.min()

-40394.43510378474

In [31]:
X /= 79483.0
X_test /= 79483.0

In [32]:
%%time

train_oof = np.zeros((X.shape[0], 10))
test_preds = 0
train_oof.shape

n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(X)):
    print("Fitting fold", jj+1)
    train_features = X[train_index]
    train_target = Y[train_index]
    
    val_features = X[val_index]
    val_target = Y[val_index]
    
    model = SVC(kernel='rbf',C=10, probability=True)
    model.fit(train_features, train_target)
    val_pred = model.predict_proba(val_features)
    train_oof[val_index] = val_pred
    print("Fold accuracy:", accuracy_score(val_target, np.argmax(val_pred, axis=1)))
    test_preds += model.predict_proba(X_test)/n_splits
    del train_features, train_target, val_features, val_target
    gc.collect()
    
print(accuracy_score(Y, np.argmax(train_oof, axis=1)))

Fitting fold 1
Fold accuracy: 0.9775
Fitting fold 2
Fold accuracy: 0.9817857142857143
Fitting fold 3
Fold accuracy: 0.9797619047619047
Fitting fold 4
Fold accuracy: 0.9785714285714285
Fitting fold 5
Fold accuracy: 0.9807142857142858
0.9796666666666667
CPU times: user 30min 17s, sys: 160 ms, total: 30min 17s
Wall time: 30min 17s
